In [29]:
from pathlib import Path
import numpy as np
import pandas as pd
import os
import ee
from typing import Dict, List, Tuple
from tqdm import tqdm
from utils import extractROI

# Authenticate and initialize the GEE
ee.Authenticate()
ee.Initialize(project="virtual-rarity-426212-p6")

ASSET_FOLDER = "projects/virtual-rarity-426212-p6/assets/shapefiles"

class SoilMetrics:
    
    def __init__(self, asset_fldr: str):
        self.roi_lst, self.basin_names = extractROI(asset_fldr)
        self.clay_data = ee.Image("projects/soilgrids-isric/clay_mean")
        self.sand_data = ee.Image("projects/soilgrids-isric/sand_mean")
        self.silt_data = ee.Image("projects/soilgrids-isric/silt_mean")
        self.water_content = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcsat")
        self.ksat_data = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/ksat")
        
    def _computeOneSoilStat(self, roi: ee.Geometry):
        
        clay_stats = self.clay_data.clip(roi).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi, 
            scale=250,
            maxPixels=1e9
        ).getInfo()
        
        sand_stats = self.sand_data.clip(roi).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi, 
            scale=250,
            maxPixels=1e9
        ).getInfo()
        
        silt_stats = self.silt_data.clip(roi).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi, 
            scale=250,
            maxPixels=1e9
        ).getInfo()
        
        max_water_content = self.water_content.reduce(ee.Reducer.mean()).reduceRegion(
            reducer=ee.Reducer.max(),
            geometry=roi,
            scale=250,
            maxPixels=1e9
        ).getInfo()

        
        ksat_stats = self.ksat_data.reduce(ee.Reducer.mean()).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi, 
            scale=250,
            maxPixels=1e9
        ).getInfo()
        
        ksat_value = list(ksat_stats.values())[0]
        max_water_content_value = list(max_water_content.values())[0] / 10000 

        return (
            clay_stats['clay_0-5cm_mean'] / 10, 
            sand_stats['sand_0-5cm_mean'] / 10, 
            silt_stats['silt_0-5cm_mean'] / 10,
            max_water_content_value,
            ksat_value/24000
            )
        
    def computeAllSoilStats(self):
        
        table = {"clay_frac": [], "sand_frac": [], "silt_frac": [], 
                 "max_water_content": [], "hydraulic_conductivity": []}
        bar_format = '{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]'
        
        for idx, roi in enumerate(tqdm(self.roi_lst, bar_format=bar_format)):
            
            if idx == 5:
                break
            
            clay_stats, sand_stats, silt_stats, water_cnt, ksat_value = self._computeOneSoilStat(roi)
            
            table['clay_frac'].append(clay_stats)
            table['sand_frac'].append(sand_stats)
            table['silt_frac'].append(silt_stats)
            table['max_water_content'].append(water_cnt)
            table['hydraulic_conductivity'].append(ksat_value)
            
        dataframe = pd.DataFrame(table, index=self.basin_names[:5])
        return dataframe





  5%|▍         | 5/106 [00:13<04:24,  2.62s/it]

       clay_frac  sand_frac  silt_frac  max_water_content  \
11001  23.345682  38.817805  37.836453           0.502900   
11063  27.147091  32.158609  40.693808           0.463900   
11068  26.701544  32.620575  40.677890           0.491767   
11077  25.943500  32.035946  42.020544           0.481850   
11090  28.246429  43.319380  28.435229           0.483267   

       hydraulic_conductivity  
11001               11.175124  
11063                5.591396  
11068                5.245892  
11077                6.615107  
11090                9.291496  
